In [1]:
'''
# @project : SDGCN
# @Author  : plzhao
# @Software: PyCharm
'''
import numpy as np
import tensorflow.compat.v1 as tf
import os
import time
import datetime
import data_helpers
from sklearn import metrics
from models.att import Att
from models.catt import CAtt
from models.att_gcn import Att_GCN
from models.catt_gcn import CAtt_GCN_L1,CAtt_GCN_L2,CAtt_GCN_L3,CAtt_GCN_L4,CAtt_GCN_L5,CAtt_GCN_L6,CAtt_GCN_L7,CAtt_GCN_L8
from models.catt_gcn_woP import CAtt_GCN_woP
# Parameters
# ==================================================
# "Restaurants" or "laptops"
use_data = "Restaurants"
# "Att"  "CAtt"  "Att_GCN"  "CAtt_GCN_L2"  "CAtt_GCN_woP"
use_model = "CAtt_GCN_L2"

tf.app.flags.DEFINE_string('f', '', 'kernel')
datas = {"Restaurants_train": "data/data_res/Restaurants_Train.txt",
         "Restaurants_test": "data/data_res/Restaurants_Test.txt",
         "Restaurants_embedding": 'data/data_res/glove.6B.300d.txt',
         "Laptops_train": "data/data_lap/Laptops_Train.txt",
         "Laptops_test": "data/data_lap/Laptops_Test.txt",
         "Laptops_embedding": 'data/data_lap/Laptops_glove.42B.300d.txt'}
#Train model
tf.flags.DEFINE_string("which_relation", 'global', "use which relation.") #'adjacent','global','rule'
tf.flags.DEFINE_string("which_model", use_model, "Model isused.")

# Data loading params
tf.flags.DEFINE_string("which_data", use_data, "Data is used.")
tf.flags.DEFINE_string("train_file", datas[use_data+"_train"], "Train data source.")
tf.flags.DEFINE_string("test_file", datas[use_data+"_test"], "Test data source.")

#word embedding
tf.flags.DEFINE_string('embedding_file_path', datas[use_data+"_embedding"], 'embedding file')
tf.flags.DEFINE_integer('word_embedding_dim', 300, 'dimension of word embedding')

# Model Hyperparameters
tf.flags.DEFINE_float("learning_rate", 1e-3, "learning_rate (default: 1e-3)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.01, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 32, "Batch Size (default: 32)")
tf.flags.DEFINE_integer("num_epochs", 80, "Number of training epochs ")
tf.flags.DEFINE_integer("evaluate_every", 5, "Evaluate model on dev set after this many steps ")
tf.flags.DEFINE_integer("checkpoint_every", 5, "Save model after this many steps")
tf.flags.DEFINE_integer("num_checkpoints", 1, "Number of checkpoints to store")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
# FLAGS._parse_flags()
# print("\nParameters:")
# for attr, value in sorted(FLAGS.__flags.items()):
#     print("{}={}".format(attr.upper(), value))
# print("")


In [2]:
'''
read from the text file.
:return:    sen word id:[324,1413,1,41,43,0,0,0]
            sen len:[5]
            sen max len :[8]
            sen label:[0,0,1]
            target word id:[34,154,0,0]
            target len: [2]
            target max len: [4]
            targets word id :[[34,154,0,0],
                              [34,14,12,56],
                              [0,0,0,0]]
            targets num = 2
            targets len: [2,4,0]
            targets max num:[3]
            targets_relation_self = [[1,0,0],
                                     [0,1,0],
                                     [0.0.0]]
            targets_relation_cross = [[0,1,0],
                                      [1,0,0],
                                      [0.0.0]]
'''
# Data Preparation
# ==================================================
# Load data
print("Loading data...")
train_x_str,train_target_str, train_y = data_helpers.load_data_and_labels(FLAGS.train_file)
dev_x_str,dev_target_str, dev_y = data_helpers.load_data_and_labels(FLAGS.test_file)
test_x_str, test_target_str, test_y = data_helpers.load_data_and_labels(FLAGS.test_file)

#word embedding ---> x[324,1413,1,41,43,0,0,0]  y[0,1]
#word_id_mapping,such as  apple--->23 ,w2v  23---->[vector]
word_id_mapping, w2v = data_helpers.load_w2v(FLAGS.embedding_file_path, FLAGS.word_embedding_dim)
max_document_length = max([len(x.split(" ")) for x in (train_x_str + dev_x_str + test_x_str)])
max_target_length = max([len(x.split(" ")) for x in (train_target_str + dev_target_str + test_target_str)])

#The targets  ---->[[[141,23,45],[23,45,1,2],[2]], ...]
#The number of targets ----> [3, ...]
train_targets_str,train_targets_num = data_helpers.load_targets(FLAGS.train_file)
dev_targets_str,dev_targets_num = data_helpers.load_targets(FLAGS.test_file)
test_targets_str, test_targets_num = data_helpers.load_targets(FLAGS.test_file)
max_target_num = max([len(x) for x in (train_targets_str + test_targets_str)])

# sentence ---> word_id
train_x, train_x_len = data_helpers.word2id(train_x_str,word_id_mapping,max_document_length)
dev_x, dev_x_len = data_helpers.word2id(dev_x_str,word_id_mapping,max_document_length)
test_x, test_x_len = data_helpers.word2id(test_x_str,word_id_mapping,max_document_length)
# target ---> word_id
train_target, train_target_len = data_helpers.word2id(train_target_str,word_id_mapping,max_target_length)
dev_target, dev_target_len = data_helpers.word2id( dev_target_str,word_id_mapping,max_target_length)
test_target, test_target_len = data_helpers.word2id(test_target_str,word_id_mapping,max_target_length)
# targets ---> word_id
train_targets, train_targets_len = data_helpers.word2id_2(train_targets_str,word_id_mapping,max_target_length,max_target_num)
dev_targets, dev_targets_len = data_helpers.word2id_2(dev_targets_str,word_id_mapping,max_target_length,max_target_num)
test_targets, test_targets_len = data_helpers.word2id_2(test_targets_str,word_id_mapping,max_target_length,max_target_num)

#which one targets in all targets
train_target_whichone = data_helpers.get__whichtarget(train_targets_num, max_target_num)
test_target_whichone = data_helpers.get__whichtarget(test_targets_num, max_target_num)
# target position
train_target_position  = data_helpers.get_position(FLAGS.train_file,max_document_length)
test_target_position  = data_helpers.get_position(FLAGS.test_file,max_document_length)

train_targets_position  = data_helpers.get_position_2(train_target_position,train_targets_num,max_target_num)
test_targets_position  = data_helpers.get_position_2(test_target_position,test_targets_num,max_target_num)

#Relation Matrix
#use test_target to creat the relation
train_relation_self,train_relation_cross = data_helpers.get_relation(train_targets_num, max_target_num,FLAGS.which_relation)
test_relation_self, test_relation_cross = data_helpers.get_relation(test_targets_num, max_target_num,FLAGS.which_relation)
Train = {'x':train_x,                       # int32(3608, 79)       train sentences input embeddingID
         'T':train_target,                  # int32(3608, 23)       train target input embeddingID
         'Ts':train_targets,                # int32(3608, 13, 23)   train targets input embeddingID
         'x_len':train_x_len,               # int32(3608,)          train sentences input len
         'T_len':train_target_len,          # int32(3608,)          train target len
         'Ts_len': train_targets_len,       # int32(3608, 13)       train targets len
         'T_W': train_target_whichone,      # int32(3608, 13)       the ith number of all the targets
         'T_P':train_target_position,       # float32(3608, 79)
         'Ts_P': train_targets_position,    # float32(3608,13, 79)
         'R_Self': train_relation_self,     # int32(3608, 13, 13)
         'R_Cross': train_relation_cross,   # int32(3608, 13, 13)
         'y': train_y,  # int32(3608, 3)
        }
Test = { 'x':test_x,
         'T':test_target,
         'Ts':test_targets,
         'x_len':test_x_len,
         'T_len':test_target_len,
         'Ts_len': test_targets_len,
         'T_W': test_target_whichone,
         'T_P': test_target_position,
         'Ts_P': test_targets_position,
         'R_Self': test_relation_self,
         'R_Cross': test_relation_cross,
         'y': test_y,
        }
#
# batches = data_helpers.batch_iter(
#     list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)

print("Vocabulary Size: {:d}".format(len(word_id_mapping)))
print("Train/Dev/test split: {:d}/{:d}/{:d}".format(len(train_y), len(dev_y), len(test_y)))
word_embedding = w2v

Loading data...
(400003, 300)
400002 400003
load word-to-id done!
load word-to-id done!
load word-to-id done!
load word-to-id done!
load word-to-id done!
load word-to-id done!
load targets-to-id done!
load targets-to-id done!
load targets-to-id done!
Vocabulary Size: 400002
Train/Dev/test split: 3608/1120/1120


In [ ]:
def train(Train, Test, word_embedding):
    # Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=FLAGS.allow_soft_placement,
          log_device_placement=FLAGS.log_device_placement)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            model = eval(use_model)(
                sequence_length=Train['x'].shape[1],
                target_sequence_length = Train['T'].shape[1],
                targets_num_max = Train['Ts'].shape[1],
                num_classes=Train['y'].shape[1],
                word_embedding = word_embedding,
                l2_reg_lambda=FLAGS.l2_reg_lambda)

            writer = tf.summary.FileWriter("logs/LSTM_GCN3", sess.graph)

            vs = tf.trainable_variables()
            print('There are %d train_able_variables in the Graph: ' % len(vs))
            for v in vs:
                print(v)

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.train.AdamOptimizer(FLAGS.learning_rate)
            grads_and_vars = optimizer.compute_gradients(model.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Keep track of gradient values and sparsity (optional)
            grad_summaries = []
            for g, v in grads_and_vars:
                if g is not None:
                    grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                    sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                    grad_summaries.append(grad_hist_summary)
                    grad_summaries.append(sparsity_summary)
            grad_summaries_merged = tf.summary.merge(grad_summaries)

            # Output directory for models and summaries
            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", use_data,use_model))
            print("Writing to {}\n".format(out_dir))

            # Summaries for loss and accuracy
            loss_summary = tf.summary.scalar("loss", model.loss)
            acc_summary = tf.summary.scalar("accuracy", model.accuracy)

            # Train Summaries
            train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            # Test summaries
            test_summary_op = tf.summary.merge([loss_summary, acc_summary])
            test_summary_dir = os.path.join(out_dir, "summaries", "test")
            test_summary_writer = tf.summary.FileWriter(test_summary_dir, sess.graph)

            # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            # else:
            #     raise Exception('The checkpoint_dir already exists:',checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)
 
            # Write vocabulary
            # vocab_processor.save(os.path.join(out_dir, "vocab"))

            # Initialize all variables
#             sess.run(tf.global_variables_initializer())
            
            saver.restore(sess, "./runs/Restaurants/CAtt_GCN_L2/checkpoints/model-7840")
            def log(string):
                file = open('./log.txt','a+')
                file.write(string+"\n")
                file.close()
            def train_step(x_batch,T_batch,Ts_batch,x_len_batch,T_len_batch,Ts_len_batch,R_Self_batch,R_Cross_batxh,T_W_batch,T_P_batch,Ts_P_batch,y_batch):
                """
                A single training step
                """
                feed_dict = {
                    model.input_x: x_batch,
                    model.input_target:T_batch,
                    model.input_targets_all:Ts_batch,
                    model.sen_len:x_len_batch,
                    model.target_len:T_len_batch,
                    model.targets_all_len_a:Ts_len_batch,
                    model.relate_self:R_Self_batch,
                    model.relate_cross:R_Cross_batxh,
                    model.target_which:T_W_batch,
                    model.target_position: T_P_batch,
                    model.targets_all_position_a: Ts_P_batch,
                    model.input_y: y_batch,
                    model.dropout_keep_prob: FLAGS.dropout_keep_prob
                }
                _, step, summaries, loss, accuracy = sess.run(
                    [train_op, global_step, train_summary_op, model.loss, model.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print(vs[19].eval())
                log("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                train_summary_writer.add_summary(summaries, step)
            
            def test_step(x_batch,T_batch,Ts_batch,x_len_batch,T_len_batch,Ts_len_batch,R_Self_batch,R_Cross_batxh,T_W_batch,T_P_batch,Ts_P_batch,y_batch, summary = None,writer=None):
                """
                Evaluates model on a dev set
                """
                feed_dict = {
                    model.input_x: x_batch,
                    model.input_target:T_batch,
                    model.input_targets_all:Ts_batch,
                    model.sen_len:x_len_batch,
                    model.target_len:T_len_batch,
                    model.targets_all_len_a:Ts_len_batch,
                    model.relate_self:R_Self_batch,
                    model.relate_cross:R_Cross_batxh,
                    model.target_which: T_W_batch,
                    model.target_position: T_P_batch,
                    model.targets_all_position_a: Ts_P_batch,
                    model.input_y: y_batch,
                    model.dropout_keep_prob: 1.0
                }
                step, summaries, loss, accuracy, softmax,true_y,predictions = sess.run(
                    [global_step, summary, model.loss, model.accuracy, model.softmax,model.true_y, model.predictions],
                    feed_dict)
                F1 = metrics.f1_score(true_y, predictions, average='macro')
                time_str = datetime.datetime.now().isoformat()
                print(vs[19].eval())
                log("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy,F1))
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy,F1))
                if writer:
                    writer.add_summary(summaries, step)
                return accuracy,softmax,F1

            # Generate batches
            batches = data_helpers.batch_iter(
                list(zip(Train['x'],Train['T'],Train['Ts'],Train['x_len'], Train['T_len'], Train['Ts_len'],
                         Train['R_Self'],Train['R_Cross'],Train['T_W'],Train['T_P'],Train['Ts_P'],Train['y'])), FLAGS.batch_size, FLAGS.num_epochs)
            # Training loop. For each batch...
            train_acc, dev_acc, test_acc, train_all_softmax, test_all_softmax = [], [], [], [], []
            max_test_acc = 0
            max_test_F1_macro = 0
            for batch in batches:
                x_batch,T_batch,Ts_batch,x_len_batch,T_len_batch,Ts_len_batch,R_Self_batch,R_Cross_batxh,T_W_batch,T_P_batch,Ts_P_batch,y_batch = zip(*batch)
                train_step(x_batch,T_batch,Ts_batch,x_len_batch,T_len_batch,Ts_len_batch,R_Self_batch,R_Cross_batxh,T_W_batch,T_P_batch,Ts_P_batch,y_batch)
                current_step = tf.train.global_step(sess, global_step)

                if current_step % FLAGS.evaluate_every == 0:
                    log('\nBy now ,the max test acc is: ')
                    print('\nBy now ,the max test acc is: ', max_test_acc)
                    log('        the max F1 score is: ')
                    print('        the max F1 score is: ', max_test_F1_macro)
                    log("\nEvaluation Text:")
                    print("\nEvaluation Text:")
                    test_acc_i, test_softmax_i, test_F1_i = test_step(Test['x'],Test['T'],Test['Ts'],Test['x_len'], Test['T_len'], Test['Ts_len'],
                                                           Test['R_Self'],Test['R_Cross'],Test['T_W'],Test['T_P'],Test['Ts_P'],Test['y'], summary = test_summary_op, writer=test_summary_writer)
                    test_acc.append(test_acc_i)
                    test_all_softmax.append(test_softmax_i)
                    log('----------------------------------------------------------')
                    print('----------------------------------------------------------')
                    print("")
                if current_step % FLAGS.checkpoint_every == 0:
                    if test_acc_i>max_test_acc:
                        path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                        # 输入ckpt模型路径
#                         input_checkpoint=path
                        # 输出pb模型的路径
                        out_pb_path="./pb/frozen_model.pb"
                        # 调用freeze_graph将ckpt转为pb
#                         freeze_graph(str(path),out_pb_path)
#                         tf.train.write_graph(sess.graph_def, './pb', 'model.pb')
#                         freeze_graph.freeze_graph(
#                             input_graph='./pb/model.pb',
#                             input_saver='',
#                             input_binary=False, 
#                             input_checkpoint=path, 
#                             output_node_names='output/Variable_1:0',
#                             restore_op_name='save/restore_all',
#                             filename_tensor_name='save/Const:0',
#                             output_graph='./pb/frozen_model.pb',
#                             clear_devices=False,
#                             initializer_nodes=''
#                         )
                        log("Saved model checkpoint to {}\n".format(path)+'\n'+'->>>>>>>>>>>>>>>>>>>>>>>')
                        print("Saved model checkpoint to {}\n".format(path))
                        print('->>>>>>>>>>>>>>>>>>>>>>>')
                        max_test_step = current_step
                        max_test_acc = test_acc_i
                    if test_F1_i > max_test_F1_macro:
                        max_test_F1_macro = test_F1_i
            log('max_test_step: ' + str(max_test_step))
            print('max_test_step: ', max_test_step)
            log('max_test_acc: ' + str(max_test_acc))
            print('max_test_acc: ', max_test_acc)
            log('max_test_F1_macro: ' + str(max_test_F1_macro))
            print('max_test_F1_Rv8xCxELmacro: ', max_test_F1_macro)
    return train_acc, dev_acc, max_test_acc,max_test_F1_macro,max_test_step, train_all_softmax, test_all_softmax


if __name__ == '__main__':
	#模型训练
	train_acc, dev_acc, max_test_acc,max_test_F1_macro,max_test_step, train_all_softmax, test_all_softmax = train(Train, Test, word_embedding)

embedding_size 300


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


LOADED Att-GCN!
There are 20 train_able_variables in the Graph: 
<tf.Variable 'bi-lstm-sentence/fw/lstm_cell/kernel:0' shape=(600, 1200) dtype=float32>
<tf.Variable 'bi-lstm-sentence/fw/lstm_cell/bias:0' shape=(1200,) dtype=float32>
<tf.Variable 'bi-lstm-sentence/bw/lstm_cell/kernel:0' shape=(600, 1200) dtype=float32>
<tf.Variable 'bi-lstm-sentence/bw/lstm_cell/bias:0' shape=(1200,) dtype=float32>
<tf.Variable 'Bi-LSTM_targets/bi-lstm-targets/fw/lstm_cell/kernel:0' shape=(600, 1200) dtype=float32>
<tf.Variable 'Bi-LSTM_targets/bi-lstm-targets/fw/lstm_cell/bias:0' shape=(1200,) dtype=float32>
<tf.Variable 'Bi-LSTM_targets/bi-lstm-targets/bw/lstm_cell/kernel:0' shape=(600, 1200) dtype=float32>
<tf.Variable 'Bi-LSTM_targets/bi-lstm-targets/bw/lstm_cell/bias:0' shape=(1200,) dtype=float32>
<tf.Variable 'Attention-targets_all2sentence/att_w_tar:0' shape=(600, 600) dtype=float32>
<tf.Variable 'Attention-targets_all2sentence/att_w_sen:0' shape=(600, 600) dtype=float32>
<tf.Variable 'GCN_layer

/root/Desktop/WHUCW/SDGCN/data_helpers.py:82: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  data = np.array(data)


[-0.01147723 -0.35723147 -0.03513337]
2022-04-05T19:00:03.259563: step 7840, loss 0.142668, acc 0.96875
[-0.01109824 -0.35766062 -0.03493931]
2022-04-05T19:00:04.558512: step 7841, loss 0.0996971, acc 0.9375
[-0.01085773 -0.35806796 -0.03462484]
2022-04-05T19:00:04.897530: step 7842, loss 0.135756, acc 0.96875
[-0.01060663 -0.35842443 -0.03437061]
2022-04-05T19:00:05.317660: step 7843, loss 0.0351788, acc 1
[-0.01037621 -0.35873887 -0.03413593]
2022-04-05T19:00:05.733580: step 7844, loss 0.0125427, acc 1

By now ,the max test acc is:  0
        the max F1 score is:  0

Evaluation Text:
[-0.01037621 -0.35873887 -0.03413593]
2022-04-05T19:01:42.242530: step 7845, loss 0.900104, acc 0.799107
----------------------------------------------------------

Saved model checkpoint to /root/Desktop/WHUCW/SDGCN/runs/Restaurants/CAtt_GCN_L2/checkpoints/model-7845

->>>>>>>>>>>>>>>>>>>>>>>
[-0.01020492 -0.3589296  -0.03396063]
2022-04-05T19:01:50.533432: step 7845, loss 0.105062, acc 0.9375
[-0.01005

...

[ 0.03690564 -0.08466412  0.01233575]
2022-04-05T19:37:14.497251: step 12637, loss 0.102878, acc 0.9375
[ 0.03716171 -0.08486775  0.01228951]
2022-04-05T19:37:14.807113: step 12638, loss 0.0928511, acc 0.9375
[ 0.03743476 -0.08504807  0.01220321]
2022-04-05T19:37:15.151811: step 12639, loss 0.0128393, acc 1

By now ,the max test acc is:  0.8044643
        the max F1 score is:  0.7221412436602309

Evaluation Text:
[ 0.03743476 -0.08504807  0.01220321]
2022-04-05T19:37:15.571525: step 12640, loss 1.19898, acc 0.772321
----------------------------------------------------------

[ 0.0375704  -0.08522467  0.01225441]
2022-04-05T19:37:15.997248: step 12640, loss 0.04396, acc 0.96875
[ 0.03772131 -0.08535496  0.01224509]
2022-04-05T19:37:16.290466: step 12641, loss 0.0322801, acc 1
[ 0.03775265 -0.08542681  0.012302  ]
2022-04-05T19:37:16.597481: step 12642, loss 0.0287437, acc 1
[ 0.03747142 -0.08533227  0.01252007]
2022-04-05T19:37:16.881808: step 12643, loss 0.368121, acc 0.9375
[ 0.037215

[ 0.03779798 -0.08520643  0.01295724]
2022-04-05T19:37:38.104635: step 12690, loss 1.30186, acc 0.765179
----------------------------------------------------------

[ 0.0378162  -0.08527689  0.01302621]
2022-04-05T19:37:38.454049: step 12690, loss 0.00529151, acc 1
[ 0.03775601 -0.08535002  0.01317865]
2022-04-05T19:37:38.821125: step 12691, loss 0.0536794, acc 0.96875
[ 0.0377176  -0.08536751  0.01325456]
2022-04-05T19:37:39.187109: step 12692, loss 0.0249828, acc 1
[ 0.03795274 -0.08545706  0.01311767]
2022-04-05T19:37:39.609406: step 12693, loss 0.109525, acc 0.9375
[ 0.03831489 -0.08542318  0.01273081]
2022-04-05T19:37:40.024276: step 12694, loss 0.122965, acc 0.9375

By now ,the max test acc is:  0.8044643
        the max F1 score is:  0.7221412436602309

Evaluation Text:
[ 0.03831489 -0.08542318  0.01273081]
2022-04-05T19:37:40.449959: step 12695, loss 1.34002, acc 0.763393
----------------------------------------------------------

[ 0.03863322 -0.08538083  0.01238091]
2022-04-0

[ 0.04831744 -0.08808681  0.00591028]
2022-04-05T19:38:03.084296: step 12742, loss 0.0759229, acc 0.96875
[ 0.04847792 -0.08839133  0.00606022]
2022-04-05T19:38:03.440821: step 12743, loss 0.0379913, acc 1
[ 0.04861128 -0.08866288  0.00620606]
2022-04-05T19:38:03.845117: step 12744, loss 0.0054617, acc 1

By now ,the max test acc is:  0.8044643
        the max F1 score is:  0.7221412436602309

Evaluation Text:
[ 0.04861128 -0.08866288  0.00620606]
2022-04-05T19:38:04.288529: step 12745, loss 1.11214, acc 0.771429
----------------------------------------------------------

[ 0.04872225 -0.0889161   0.00635721]
2022-04-05T19:38:04.654096: step 12745, loss 0.0217664, acc 1
[ 0.04889227 -0.08909891  0.00637836]
2022-04-05T19:38:05.085224: step 12746, loss 0.0542671, acc 0.96875
[ 0.04903443 -0.08929577  0.00644211]
2022-04-05T19:38:05.570265: step 12747, loss 0.0157121, acc 1
[ 0.04900642 -0.08945368  0.00664393]
2022-04-05T19:38:05.978452: step 12748, loss 0.0910114, acc 0.96875
[ 0.04889

[ 0.04097982 -0.08575978  0.01213952]
2022-04-05T19:38:27.230898: step 12795, loss 1.12427, acc 0.771429
----------------------------------------------------------

[ 0.04049088 -0.08515971  0.01207714]
2022-04-05T19:38:27.584744: step 12795, loss 0.243992, acc 0.9375
[ 0.03988238 -0.08468537  0.01225939]
2022-04-05T19:38:27.947924: step 12796, loss 0.0742367, acc 0.96875
[ 0.03935423 -0.08429836  0.01244378]
2022-04-05T19:38:28.312380: step 12797, loss 0.015094, acc 1
[ 0.03888771 -0.08396385  0.01261583]
2022-04-05T19:38:28.671329: step 12798, loss 0.00702489, acc 1
[ 0.03847979 -0.08379834  0.01289088]
2022-04-05T19:38:29.034771: step 12799, loss 0.369374, acc 0.9375

By now ,the max test acc is:  0.8044643
        the max F1 score is:  0.7221412436602309

Evaluation Text:
[ 0.03847979 -0.08379834  0.01289088]
2022-04-05T19:38:29.475817: step 12800, loss 0.972511, acc 0.770536
----------------------------------------------------------

[ 0.03819793 -0.08373893  0.01313965]
2022-04-0

[ 0.03914602 -0.08701505  0.01618192]
2022-04-05T19:38:52.651503: step 12847, loss 0.266468, acc 0.9375
[ 0.03886883 -0.08636969  0.01585454]
2022-04-05T19:38:53.006211: step 12848, loss 0.242481, acc 0.9375
[ 0.03866645 -0.08580204  0.01552635]
2022-04-05T19:38:53.345767: step 12849, loss 0.0396178, acc 1

By now ,the max test acc is:  0.8044643
        the max F1 score is:  0.7221412436602309

Evaluation Text:
[ 0.03866645 -0.08580204  0.01552635]
2022-04-05T19:38:53.774657: step 12850, loss 1.0102, acc 0.774107
----------------------------------------------------------

[ 0.03844463 -0.08513025  0.01511612]
2022-04-05T19:38:54.136276: step 12850, loss 0.127846, acc 0.9375
[ 0.03826957 -0.08457939  0.01477581]
2022-04-05T19:38:54.569081: step 12851, loss 0.0205061, acc 1
[ 0.03821876 -0.084114    0.01439193]
2022-04-05T19:38:54.944948: step 12852, loss 0.0484036, acc 0.96875
[ 0.03816946 -0.08369336  0.01405005]
2022-04-05T19:38:55.308700: step 12853, loss 0.00500769, acc 1
[ 0.03815

In [ ]:
from tensorflow.python.framework import graph_util
def freeze_graph(input_checkpoint,output_graph):
    '''
    :param input_checkpoint:
    :param output_graph: PB模型保存路径
    :return:
    '''
    # checkpoint = tf.train.get_checkpoint_state(model_folder) #检查目录下ckpt文件状态是否可用
    # input_checkpoint = checkpoint.model_checkpoint_path #得ckpt文件路径
 
    # 指定输出的节点名称,该节点名称必须是原模型中存在的节点
    output_node_names = "input_x,input_x_1,input_x_2,sen_len,target_len," +\
    "targets_all_len/targets_all_len,relate_self,relate_cross,"+\
    "which_position,target_position,targets_all_position/targets_all_position,"+\
    "input_y,dropout_keep_prob,output/softmax"
    
    tf.disable_v2_behavior()
    saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=True)
 
    with tf.Session() as sess:
        saver.restore(sess, input_checkpoint) #恢复图并得到数据
        output_graph_def = graph_util.convert_variables_to_constants(  # 模型持久化，将变量值固定
            sess=sess,
            input_graph_def=sess.graph_def,# 等于:sess.graph_def
            output_node_names=output_node_names.split(","))# 如果有多个输出节点，以逗号隔开
 
        with tf.gfile.GFile(output_graph, "wb") as f: #保存模型
            f.write(output_graph_def.SerializeToString()) #序列化输出
        print("%d ops in the final graph." % len(output_graph_def.node)) #得到当前图有几个操作节点
freeze_graph("./runs/Restaurants/CAtt_GCN_L2/checkpoints/model-4345","./pb/model.pb")

In [ ]:
w2v_file = 'data/data_res/300_rest14_embedding_matrix.pkl'
from gensim.models import Word2Vec
model = Word2Vec.load(w2v_file)